Without PyTorch

In [23]:
import numpy as np

# model: y = w*x = 2*x

# Initialisation
x = np.array([1,2,3,4], dtype=np.float32)
y = np.array([2,4,6,8], dtype=np.float32)
w = 0.0

# Forward pass
def forw_pass(x,w):
    y_hat = w*x
    return y_hat

# Loss MSE
def loss(y, y_hat):
    return ((y_hat - y)**2).mean()

# Gradient
def gradient(x,y,y_hat):
    return np.dot(2*x,y_hat-y).mean()

alpha = .01
num_iteration = 15

for epoch in range(num_iteration):
    y_hat = forw_pass(x,w)
    l = loss(y,y_hat)
    dL_dw = gradient(x,y,y_hat)
    
    w = w - alpha * dL_dw
    print(f"Epoch:{epoch+1}  Loss:{l:.8f}  weight:{w:.5f}")

print(f"\nPrediction after training: f(5) = {forw_pass(5,w):.3f}")
    

Epoch:1  Loss:30.00000000  weight:1.20000
Epoch:2  Loss:4.79999924  weight:1.68000
Epoch:3  Loss:0.76800019  weight:1.87200
Epoch:4  Loss:0.12288000  weight:1.94880
Epoch:5  Loss:0.01966083  weight:1.97952
Epoch:6  Loss:0.00314570  weight:1.99181
Epoch:7  Loss:0.00050332  weight:1.99672
Epoch:8  Loss:0.00008053  weight:1.99869
Epoch:9  Loss:0.00001288  weight:1.99948
Epoch:10  Loss:0.00000206  weight:1.99979
Epoch:11  Loss:0.00000033  weight:1.99992
Epoch:12  Loss:0.00000005  weight:1.99997
Epoch:13  Loss:0.00000001  weight:1.99999
Epoch:14  Loss:0.00000000  weight:1.99999
Epoch:15  Loss:0.00000000  weight:2.00000

Prediction after training: f(5) = 10.000


Using Autograd of PyTorch

In [7]:
import torch

# model: y = w*x = 2*x

# Initialisation
x = torch.tensor([1,2,3,4], dtype=torch.float32)
y = torch.tensor([2,4,6,8], dtype=torch.float32)
w = torch.tensor(0.0, requires_grad=True)

# Forward pass
def forw_pass(x,w):
    y_hat = w*x
    return y_hat

# Loss MSE
def loss(y, y_hat):
    return ((y_hat - y)**2).mean()

# Gradient calc done by autograd

alpha = .01
num_iteration = 15

for epoch in range(num_iteration):
    y_hat = forw_pass(x,w)
    l = loss(y,y_hat)
    l.backward()
    
    with torch.no_grad():
        w = w - alpha * w.grad
        
    w.grad.zero_()
    print(f"Epoch:{epoch+1}  Loss:{l:.8f}  weight:{w:.5f}")

print(f"\nPrediction after training: f(5) = {forw_pass(5,w):.3f}")
    

AttributeError: 'NoneType' object has no attribute 'zero_'